**Ejercicio 1: Prepara tu dataset favorito**

In [ ]:
# Instalar paquetes necesarios en Google Colab
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00


In [ ]:
from importlib.metadata import version
import re
import tiktoken
import torch


In [ ]:


# Imprimir versiones de los paquetes
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

uploaded = files.upload()  # Permite subir archivos manualmente
data_path = list(uploaded.keys())[0]  # Toma el primer archivo subido

# Cargar el archivo de texto
with open(data_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print(raw_text[:99])

# Tokenización simple con expresiones regulares
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip().lower() for item in preprocessed if item.strip()]
print(preprocessed[:38])
print("Number of tokens:", len(preprocessed))

# Construcción del vocabulario con un token <UNK> para palabras desconocidas
all_words = sorted(set(preprocessed))
vocab_size = len(all_words) + 1  # +1 para incluir el token <UNK>
vocab = {token: integer for integer, token in enumerate(all_words)}
vocab["<UNK>"] = len(vocab)  # Agregar el token especial <UNK>

print("Vocabulary size:", vocab_size)

# Implementación de un tokenizador con manejo de palabras desconocidas
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
        self.unknown_token = "<UNK>"

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip().lower() for item in preprocessed if item.strip()]
        ids = [self.str_to_int.get(s, self.str_to_int["<UNK>"]) for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str.get(i, self.unknown_token) for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

# Prueba del tokenizador
tokenizer = SimpleTokenizerV1(vocab)
text_sample = "Hello, world! This is a test."
encoded = tokenizer.encode(text_sample)
print("Encoded:", encoded)
print("Decoded:", tokenizer.decode(encoded))

# Tokenización con tiktoken
tiktokenizer = tiktoken.get_encoding("gpt2")
integers = tiktokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
print("Tiktoken encoded sample:", integers[:20])

# Implementar ventana deslizante para el muestreo de datos
def create_dataloader_v1(text, batch_size=8, max_length=4, stride=4):
    tokens = tokenizer.encode(text)
    inputs, targets = [], []

    for i in range(0, len(tokens) - max_length, stride):
        inputs.append(tokens[i:i + max_length])
        targets.append(tokens[i + 1:i + max_length + 1])

    return torch.tensor(inputs), torch.tensor(targets)

# Aplicar la ventana deslizante
inputs, targets = create_dataloader_v1(raw_text)
print("Inputs:", inputs[:3])
print("Targets:", targets[:3])

torch version: 2.6.0+cu124
tiktoken version: 0.9.0


**Ejercicio 2: Generar texto**

In [ ]:
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 1️⃣ Cargar el modelo y el tokenizador preentrenados de Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 2️⃣ Preparar el texto de entrada
texto_entrada = "Artificial intelligence is"
tokens = tokenizer.encode(texto_entrada, return_tensors="pt")  # Convertir a tensor

# 3️⃣ Generar texto
with torch.no_grad():  # Desactivar gradientes para hacer inferencia
    output_tokens = model.generate(tokens, max_length=50, do_sample=True, top_k=50)

# 4️⃣ Decodificar el texto generado
texto_generado = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# 5️⃣ Imprimir resultado
print("Texto generado:")
print(texto_generado)





tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Texto generado:
Artificial intelligence is still at a nascent stage and many fields are being worked on in a way that many researchers are starting to catch up with the next generation.

However, one of the most notable breakthroughs that have emerged in artificial intelligence from


**Ejercicio 3: Generar texto usando el modelo pre-entrenado**

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 1️⃣ Cargar el modelo y el tokenizador preentrenados de Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configurar el token de padding para evitar advertencias
tokenizer.pad_token = tokenizer.eos_token

# 2️⃣ Texto de entrada
texto_entrada = "The future of artificial intelligence is"

# 3️⃣ Tokenizar y convertir a tensor
tokens = tokenizer.encode(texto_entrada, return_tensors="pt")

# 4️⃣ Generar texto con el modelo
with torch.no_grad():
    output_tokens = model.generate(
        tokens,
        max_length=100,  # Número máximo de tokens en el texto generado
        do_sample=True,  # Permite variabilidad en la generación
        top_k=50,  # Controla la aleatoriedad en la generación
        pad_token_id=tokenizer.eos_token_id  # Evita advertencias
    )

# 5️⃣ Convertir IDs de tokens a texto y limpiar salida
texto_generado = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
texto_generado = texto_generado[len(texto_entrada):].strip()  # Quitar texto de entrada

# 6️⃣ Mostrar resultado
print("Texto generado:")
print(texto_generado)


Texto generado:
currently unknown. But now, we know that there is something that needs to be addressed, and that could well end up being a problem for the foreseeable future.

Why is artificial intelligence so important? The question that gets a huge amount of attention is how this technology will be used. That question was raised last year in a talk given by Dr. Michael L. Cohen, a psychiatrist of behavioral science and director of the Institute for Cognition and Brain Networks at MIT


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 1️⃣ Cargar el modelo pre-entrenado y el tokenizador
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configurar el token de padding para evitar advertencias
tokenizer.pad_token = tokenizer.eos_token

# 2️⃣ Definir un nuevo texto de entrada
texto_entrada = "Technology is evolving at an unprecedented rate"

# 3️⃣ Tokenizar y convertir a tensor
tokens = tokenizer.encode(texto_entrada, return_tensors="pt")

# 4️⃣ Generar texto con el modelo pre-entrenado
with torch.no_grad():
    output_tokens = model.generate(
        tokens,
        max_length=100,  # Número máximo de tokens en el texto generado
        do_sample=True,  # Permite variabilidad en la generación
        top_k=50,  # Controla la aleatoriedad en la generación
        pad_token_id=tokenizer.eos_token_id  # Evita advertencias
    )

# 5️⃣ Convertir IDs de tokens a texto y limpiar la salida
texto_generado = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
texto_generado = texto_generado[len(texto_entrada):].strip()  # Quitar el texto de entrada

# 6️⃣ Mostrar resultado
print("Texto generado:")
print(texto_generado)


Texto generado:
and its business model relies on a highly-effective mix of the top-end and the small- to medium range of companies and industries. We expect there will be significant growth in large and small companies as they grow their business. We believe that this growth, along with our strategic and strategic partnerships with several leading market segments, will provide an opportunity to accelerate our transformation efforts by engaging in large scale, global operations with our existing and potential customers and by engaging with third


**Ejercicio 4: Cargar el modelo pre-entrenado y generar un nuevo texto**

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 1️⃣ Cargar el modelo y el tokenizador preentrenados
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configurar el token de padding para evitar advertencias
tokenizer.pad_token = tokenizer.eos_token

# 2️⃣ Definir el texto de entrada
texto_entrada = "The future of artificial intelligence"

# 3️⃣ Tokenizar y convertir a tensor
tokens = tokenizer.encode(texto_entrada, return_tensors="pt")

# 4️⃣ Generar un nuevo texto con el modelo
with torch.no_grad():
    output_tokens = model.generate(
        tokens,
        max_length=100,  # Máximo número de tokens generados
        do_sample=True,  # Permite variabilidad en la generación
        top_k=50,  # Controla la aleatoriedad
        pad_token_id=tokenizer.eos_token_id  # Evita advertencias
    )

# 5️⃣ Convertir IDs de tokens a texto y limpiar la salida
texto_generado = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
texto_generado = texto_generado[len(texto_entrada):].strip()  # Eliminar texto de entrada

# 6️⃣ Mostrar resultado
print("Texto generado:")
print(texto_generado)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Texto generado:
, which is what we've been talking about for the past 10 years now, has been extremely promising. If we can build AI systems from scratch without being able to do so at the scale that we need, what exactly does that mean for our future of AI?

The technology industry is already pushing back on this, with companies like Intel stating that we need to see "the capabilities of real-world, intelligent AI at the level of real people before our day arrives


In [3]:
!pip install datasets transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

**Ejercicio 5: Entrena el LLM en un texto de tu elección**


In [8]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

# 1️⃣ Cargar el modelo y el tokenizador preentrenados
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configurar el token de padding
tokenizer.pad_token = tokenizer.eos_token

# 2️⃣ Crear el dataset de entrenamiento (puedes cambiar el texto)
texto_entrenamiento = """Artificial intelligence is transforming the world.
Machine learning and deep learning are driving innovations in various industries,
including healthcare, finance, and robotics."""

# Convertir el texto en un DataFrame para usarlo con Hugging Face Dataset
data = {"text": [texto_entrenamiento]}
df = pd.DataFrame(data)

# Convertir a dataset de Hugging Face
dataset = Dataset.from_pandas(df)

# Tokenizar el dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

dataset = dataset.map(tokenize_function, batched=True)

# 3️⃣ Configurar el entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
)

# Data Collator para el entrenamiento
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No usamos enmascaramiento porque GPT-2 es un modelo de autoregresión
)

# 4️⃣ Entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# 5️⃣ Probar el modelo con texto generado
texto_entrada = "The future of AI"
tokens = tokenizer.encode(texto_entrada, return_tensors="pt")

with torch.no_grad():
    output_tokens = model.generate(
        tokens,
        max_length=100,
        do_sample=True,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id
    )

# Convertir tokens a texto
texto_generado = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print("\nTexto generado después del entrenamiento:")
print(texto_generado)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

<ipython-input-8-858f7ca05440>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss



Texto generado después del entrenamiento:
The future of AI technology is to build and support new artificial intelligence, robotics, and biotechnology.

The company is not interested in getting better at artificial intelligence at the level of the US Army and the US military. The real problem is, "how will they compete with the other AI startups on the level of the US Army? Will they grow their business and keep pace over the long- and not get into the US Army?"

"And even just for the US Army is the


In [10]:
from transformers import pipeline

### 1️⃣ CLASIFICACIÓN DE SENTIMIENTO ###
cls_pipeline = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")

texto_sentimiento = "Estoy muy emocionado por aprender sobre inteligencia artificial."

resultado_sentimiento = cls_pipeline(texto_sentimiento)
print("\n🔹 Clasificación de sentimiento:")
print(f"Texto: {texto_sentimiento}")
print(f"Clasificación: {resultado_sentimiento[0]['label']} - Confianza: {resultado_sentimiento[0]['score']:.4f}")

### 2️⃣ RECONOCIMIENTO DE ENTIDADES (NER) ###
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")

texto_ner = "Elon Musk es el CEO de Tesla y SpaceX."

resultados_ner = ner_pipeline(texto_ner)
print("\n🔹 Reconocimiento de Entidades (NER):")
for resultado in resultados_ner:
    print(f"Token: {resultado['word']} - Etiqueta: {resultado['entity']} - Confianza: {resultado['score']:.4f}")

### 3️⃣ RESPONDER PREGUNTAS ###
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

contexto = "El Quijote fue escrito por Miguel de Cervantes en el siglo XVII."
pregunta = "¿Quién escribió El Quijote?"

respuesta = qa_pipeline(question=pregunta, context=contexto)
print("\n🔹 Respuesta a Pregunta:")
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {respuesta['answer']} - Confianza: {respuesta['score']:.4f}")

### 4️⃣ RESUMIR UN TEXTO LARGO ###
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

texto_largo = """
La inteligencia artificial es una rama de la informática que se centra en crear sistemas que pueden realizar tareas
que normalmente requieren inteligencia humana. Estas tareas incluyen el reconocimiento de voz, la toma de decisiones,
el procesamiento del lenguaje natural y la visión por computadora.
"""

resumen = summarizer(texto_largo, max_length=50, min_length=20, do_sample=False)
print("\n🔹 Resumen del Texto:")
print(resumen[0]['summary_text'])

### 5️⃣ GENERAR TEXTO ###
generador = pipeline("text-generation", model="gpt2")

texto_inicial = "La inteligencia artificial está cambiando"
texto_generado = generador(texto_inicial, max_length=100, num_return_sequences=1)

print("\n🔹 Generación de Texto:")
print(texto_generado[0]['generated_text'])


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



🔹 Resumen del Texto:
La inteligencia artificial es una rama de la informática. Se centra en crear sistemas que pueden realizar tareas que normalmente requieren inteligence


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔹 Generación de Texto:
La inteligencia artificial está cambiando todo las primores a los español.

"Sì casa por algo y sus y español con el jugar," she declared at length. "La casa puedo y que estun con sus y español que si el donde el suerte."

"La casa y español que sus suerte, llegarse c
